<a href="https://colab.research.google.com/github/ralpheeet/A1-Introduccion-a-Bases-de-Datos-Santander2021/blob/main/Bayesiano_ingenuo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Un ejemplo de clasificación en el área médica

## Con (estimador máximo-verosímil) EMV

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Datos de entrenamiento
data = {
    'Temperatura': [36.5, 37.2, 36.8, 37.5, 36.9, 37.0, 35.3, 37.1, 34.9, 38.1, 36.7, 36.5, 37.22],
    'Presion_Arterial': [120, 140, 130, 150, 125, 145, 129, 141, 153, 151, 125, 158, 160],
    'Comorbilidad': ['Obesidad', 'Obesidad', 'Ansiedad', 'Asma', 'Asma', 'Arritmia', 'Ansiedad', 'Obesidad',
                     'Ansiedad', 'Obesidad', 'Ansiedad', 'Asma', 'Arritmia'],
    'Clasificacion': ['S', 'E', 'S', 'E', 'S', 'E', 'S', 'E', 'S', 'E', 'S', 'E', 'E']
}

# Crear el DataFrame
df = pd.DataFrame(data)

# Codificar variables categóricas
label_encoder_comorbilidad = LabelEncoder()
df['Comorbilidad'] = label_encoder_comorbilidad.fit_transform(df['Comorbilidad'])
label_encoder_clasificacion = LabelEncoder()
df['Clasificacion'] = label_encoder_clasificacion.fit_transform(df['Clasificacion'])

# Dividir las características (X) y el objetivo (y)
X = df[['Temperatura', 'Presion_Arterial', 'Comorbilidad']].values
y = df['Clasificacion'].values

# Visualizar estadísticas
print("Estadísticas de las características por clase:")
for c in nb_emv.classes:
    print(f"\nClase {label_encoder_clasificacion.inverse_transform([c])[0]}:")
    print(f"Media (Temperatura, Presión, Comorbilidad): {nb_emv.feature_stats[c]['mean']}")
    print(f"Varianza (Temperatura, Presión, Comorbilidad): {nb_emv.feature_stats[c]['var']}")

print( )
# Implementación personalizada de Naive Bayes con EMV
class NaiveBayesEMV:
    def fit(self, X, y):
        self.classes, counts = np.unique(y, return_counts=True)
        self.priors = counts / len(y)  # P(y)
        self.feature_stats = {}

        for c in self.classes:
            X_c = X[y == c]
            self.feature_stats[c] = {
                'mean': X_c.mean(axis=0),
                'var': X_c.var(axis=0),  # Varianza no ajustada
            }

    def _gaussian_prob(self, x, mean, var):
        coeff = 1.0 / np.sqrt(2.0 * np.pi * var)
        exponent = np.exp(-((x - mean) ** 2) / (2 * var))
        return coeff * exponent

    def predict_proba(self, X):
        probabilities = []
        for x in X:
            class_probs = []
            for c in self.classes:
                prior = self.priors[c]
                likelihoods = np.prod(
                    self._gaussian_prob(x, self.feature_stats[c]['mean'], self.feature_stats[c]['var'])
                )
                class_probs.append(prior * likelihoods)
            print(f"Probabilidades antes de normalizar para {x}: {class_probs}")
            total = np.sum(class_probs)
            normalized_probs = [p / total for p in class_probs]  # Normalizar
            probabilities.append(normalized_probs)
        return np.array(probabilities)

    def predict(self, X):
        probas = self.predict_proba(X)
        return np.argmax(probas, axis=1)

# Entrenar el modelo personalizado con EMV
nb_emv = NaiveBayesEMV()
nb_emv.fit(X, y)

# Nuevos datos para predecir
nuevos_datos = np.array([
    [35.9, 143, label_encoder_comorbilidad.transform(['Obesidad'])[0]],
    [36.0, 140, label_encoder_comorbilidad.transform(['Asma'])[0]],
    [37.2, 125, label_encoder_comorbilidad.transform(['Asma'])[0]],
    [36.4, 120, label_encoder_comorbilidad.transform(['Arritmia'])[0]],
    [36.8, 162, label_encoder_comorbilidad.transform(['Obesidad'])[0]]
])

# Calcular probabilidades y predicciones
probas = nb_emv.predict_proba(nuevos_datos)
predicciones_emv = nb_emv.predict(nuevos_datos)

# Mostrar los resultados
for i, (pred_num, prob) in enumerate(zip(predicciones_emv, probas)):
    pred_clase = label_encoder_clasificacion.inverse_transform([pred_num])[0]
    print(f"Paciente {i+1}: Clase Predicha - {pred_clase} (Numérica: {pred_num})")
    print(f"\tProbabilidad de Enfermo (E): {prob[0]:.8f}")
    print(f"\tProbabilidad de Sano (S): {prob[1]:.8f}")


Estadísticas de las características por clase:

Clase E:
Media (Temperatura, Presión, Comorbilidad): [ 37.23142857 149.28571429   2.14285714]
Varianza (Temperatura, Presión, Comorbilidad): [ 0.20478367 52.48979592  0.69387755]

Clase S:
Media (Temperatura, Presión, Comorbilidad): [ 36.18333333 130.33333333   0.83333333]
Varianza (Temperatura, Presión, Comorbilidad): [  0.61472222 113.22222222   1.47222222]

Probabilidades antes de normalizar para [ 35.9 143.    3. ]: [6.675127098620324e-05, 0.0002711199556535683]
Probabilidades antes de normalizar para [ 36. 140.   2.]: [0.0001338128526015201, 0.0011743530406326387]
Probabilidades antes de normalizar para [ 37.2 125.    2. ]: [4.468613411430369e-05, 0.0006937719377304019]
Probabilidades antes de normalizar para [ 36.4 120.    1. ]: [2.5570626909479106e-07, 0.0017225768150556794]
Probabilidades antes de normalizar para [ 36.8 162.    3. ]: [0.0010035069645186955, 5.148602687223667e-06]
Probabilidades antes de normalizar para [ 35.9 143.

## Simplificando implementar EMV con scikit-learn

In [ ]:
# Importar las bibliotecas necesarias
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder

# Datos de entrenamiento
# Convertir los datos manualmente en un DataFrame
data = {
    'Temperatura': [36.5, 37.2, 36.8, 37.5, 36.9, 37.0, 35.3, 37.1, 34.9, 38.1, 36.7, 36.5, 37.22],
    'Presion_Arterial': [120, 140, 130, 150, 125, 145, 129, 141, 153, 151, 125, 158, 160],
    'Comorbilidad': ['Obesidad', 'Obesidad', 'Ansiedad', 'Asma', 'Asma', 'Arritmia', 'Ansiedad', 'Obesidad',
                     'Ansiedad', 'Obesidad', 'Ansiedad', 'Asma', 'Arritmia'],
    'Clasificacion': ['S', 'E', 'S', 'E', 'S', 'E', 'S', 'E', 'S', 'E', 'S', 'E', 'E']
}

# Crear el DataFrame
df = pd.DataFrame(data)

# Codificar variables categóricas
label_encoder_comorbilidad = LabelEncoder()
df['Comorbilidad'] = label_encoder_comorbilidad.fit_transform(df['Comorbilidad'])
label_encoder_clasificacion = LabelEncoder()
df['Clasificacion'] = label_encoder_clasificacion.fit_transform(df['Clasificacion'])

# Dividir las características (X) y el objetivo (y)
X = df[['Temperatura', 'Presion_Arterial', 'Comorbilidad']]
y = df['Clasificacion']

# Entrenar el clasificador bayesiano ingenuo
gnb = GaussianNB()
gnb.fit(X, y)

# Nuevos datos para predecir
nuevos_datos = [
    [35.9, 143, label_encoder_comorbilidad.transform(['Obesidad'])[0]],
    [36.0, 140, label_encoder_comorbilidad.transform(['Asma'])[0]],
    [37.2, 125, label_encoder_comorbilidad.transform(['Asma'])[0]],
    [36.4, 120, label_encoder_comorbilidad.transform(['Arritmia'])[0]],
    [36.8, 162, label_encoder_comorbilidad.transform(['Obesidad'])[0]]
]

# Realizar las predicciones
predicciones = gnb.predict(nuevos_datos)
predicciones_clases = label_encoder_clasificacion.inverse_transform(predicciones)

# Mostrar los resultados
for i, pred in enumerate(predicciones_clases):
    print(f"Paciente {i+1}: Clase Predicha - {pred}")


Paciente 1: Clase Predicha - S
Paciente 2: Clase Predicha - S
Paciente 3: Clase Predicha - S
Paciente 4: Clase Predicha - S
Paciente 5: Clase Predicha - E


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


## Con estimación máxima a posteriori (MAP)

In [ ]:
# Importar las bibliotecas necesarias
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Datos de entrenamiento
# Convertir los datos manualmente en un DataFrame
data = {
    'Temperatura': [36.5, 37.2, 36.8, 37.5, 36.9, 37.0, 35.3, 37.1, 34.9, 38.1, 36.7, 36.5, 37.22],
    'Presion_Arterial': [120, 140, 130, 150, 125, 145, 129, 141, 153, 151, 125, 158, 160],
    'Comorbilidad': ['Obesidad', 'Obesidad', 'Ansiedad', 'Asma', 'Asma', 'Arritmia', 'Ansiedad', 'Obesidad',
                     'Ansiedad', 'Obesidad', 'Ansiedad', 'Asma', 'Arritmia'],
    'Clasificacion': ['S', 'E', 'S', 'E', 'S', 'E', 'S', 'E', 'S', 'E', 'S', 'E', 'E']
}

# Crear el DataFrame
df = pd.DataFrame(data)

# Codificar variables categóricas
label_encoder_comorbilidad = LabelEncoder()
df['Comorbilidad'] = label_encoder_comorbilidad.fit_transform(df['Comorbilidad'])
label_encoder_clasificacion = LabelEncoder()
df['Clasificacion'] = label_encoder_clasificacion.fit_transform(df['Clasificacion'])

# Dividir las características (X) y el objetivo (y)
X = df[['Temperatura', 'Presion_Arterial', 'Comorbilidad']]
y = df['Clasificacion']

# Implementación personalizada para MAP
class NaiveBayesMAP:
    def __init__(self, alpha=1.0):
        self.alpha = alpha

    def fit(self, X, y):
        self.classes, counts = np.unique(y, return_counts=True)
        self.priors = counts / len(y)
        self.feature_stats = {}

        for c in self.classes:
            X_c = X[y == c]
            self.feature_stats[c] = {
                'mean': X_c.mean(axis=0),
                'var': X_c.var(axis=0) + self.alpha,
            }

    def _gaussian_prob(self, x, mean, var):
        coeff = 1.0 / np.sqrt(2.0 * np.pi * var)
        exponent = np.exp(-((x - mean) ** 2) / (2 * var))
        return coeff * exponent

    def predict(self, X):
        posteriors = []
        for x in X:
            class_posteriors = []
            for c in self.classes:
                prior = np.log(self.priors[c])
                likelihoods = np.sum(
                    np.log(self._gaussian_prob(x, self.feature_stats[c]['mean'], self.feature_stats[c]['var']))
                )
                class_posteriors.append(prior + likelihoods)
            posteriors.append(self.classes[np.argmax(class_posteriors)])
        return np.array(posteriors)

# Entrenar el modelo personalizado con MAP
nb_map = NaiveBayesMAP(alpha=1.0)
X_np = X.values
y_np = y.values
nb_map.fit(X_np, y_np)

# Nuevos datos para predecir
nuevos_datos = np.array([
    [35.9, 143, label_encoder_comorbilidad.transform(['Obesidad'])[0]],
    [36.0, 140, label_encoder_comorbilidad.transform(['Asma'])[0]],
    [37.2, 125, label_encoder_comorbilidad.transform(['Asma'])[0]],
    [36.4, 120, label_encoder_comorbilidad.transform(['Arritmia'])[0]],
    [36.8, 162, label_encoder_comorbilidad.transform(['Obesidad'])[0]]
])

# Realizar las predicciones
predicciones_map = nb_map.predict(nuevos_datos)
predicciones_clases_map = label_encoder_clasificacion.inverse_transform(predicciones_map)

# Mostrar los resultados
for i, pred in enumerate(predicciones_clases_map):
    print(f"Paciente {i+1}: Clase Predicha (MAP) - {pred}")
    #print(f"\tProbabilidad de Sano (S): {proba[0]:.8f}")
    #print(f"\tProbabilidad de Enfermo (E): {proba[1]:.8f}")


Paciente 1: Clase Predicha (MAP) - E
Paciente 2: Clase Predicha (MAP) - E
Paciente 3: Clase Predicha (MAP) - S
Paciente 4: Clase Predicha (MAP) - S
Paciente 5: Clase Predicha (MAP) - E


In [ ]:
# Importar las bibliotecas necesarias
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Datos de entrenamiento
data = {
    'Temperatura': [36.5, 37.2, 36.8, 37.5, 36.9, 37.0, 35.3, 37.1, 34.9, 38.1, 36.7, 36.5, 37.22],
    'Presion_Arterial': [120, 140, 130, 150, 125, 145, 129, 141, 153, 151, 125, 158, 160],
    'Comorbilidad': ['Obesidad', 'Obesidad', 'Ansiedad', 'Asma', 'Asma', 'Arritmia', 'Ansiedad', 'Obesidad',
                     'Ansiedad', 'Obesidad', 'Ansiedad', 'Asma', 'Arritmia'],
    'Clasificacion': ['S', 'E', 'S', 'E', 'S', 'E', 'S', 'E', 'S', 'E', 'S', 'E', 'E']
}

# Crear el DataFrame
df = pd.DataFrame(data)

# Codificar variables categóricas
label_encoder_comorbilidad = LabelEncoder()
df['Comorbilidad'] = label_encoder_comorbilidad.fit_transform(df['Comorbilidad'])
label_encoder_clasificacion = LabelEncoder()
df['Clasificacion'] = label_encoder_clasificacion.fit_transform(df['Clasificacion'])

# Dividir las características (X) y el objetivo (y)
X = df[['Temperatura', 'Presion_Arterial', 'Comorbilidad']].values
y = df['Clasificacion'].values

# Implementación personalizada para MAP
class NaiveBayesMAP:
    def __init__(self, alpha=1.0):
        self.alpha = alpha

    def fit(self, X, y):
        self.classes, counts = np.unique(y, return_counts=True)
        self.priors = counts / len(y)
        self.feature_stats = {}

        for c in self.classes:
            X_c = X[y == c]
            self.feature_stats[c] = {
                'mean': X_c.mean(axis=0),
                'var': X_c.var(axis=0) + self.alpha,
            }

    def _gaussian_prob(self, x, mean, var):
        coeff = 1.0 / np.sqrt(2.0 * np.pi * var)
        exponent = np.exp(-((x - mean) ** 2) / (2 * var))
        return coeff * exponent

    def predict_proba(self, X):
        probabilities = []
        for x in X:
            class_probs = []
            for c in self.classes:
                prior = np.log(self.priors[c])
                likelihoods = np.sum(
                    np.log(self._gaussian_prob(x, self.feature_stats[c]['mean'], self.feature_stats[c]['var']))
                )
                class_probs.append(np.exp(prior + likelihoods))  # Convertir de log-probabilidad a probabilidad
            print(f"Probabilidades antes de normalizar para {x}: {class_probs}")
            total = np.sum(class_probs)
            normalized_probs = [p / total for p in class_probs]  # Normalizar
            probabilities.append(normalized_probs)
        return np.array(probabilities)

    def predict(self, X):
        probas = self.predict_proba(X)
        return np.argmax(probas, axis=1)

# Entrenar el modelo personalizado con MAP
nb_map = NaiveBayesMAP(alpha=1.0)
nb_map.fit(X, y)

# Nuevos datos para predecir
nuevos_datos = np.array([
    [35.9, 143, label_encoder_comorbilidad.transform(['Obesidad'])[0]],
    [36.0, 140, label_encoder_comorbilidad.transform(['Asma'])[0]],
    [37.2, 125, label_encoder_comorbilidad.transform(['Asma'])[0]],
    [36.4, 120, label_encoder_comorbilidad.transform(['Arritmia'])[0]],
    [36.8, 162, label_encoder_comorbilidad.transform(['Obesidad'])[0]]
])

# Calcular probabilidades y predicciones
probas = nb_map.predict_proba(nuevos_datos)
predicciones_map = nb_map.predict(nuevos_datos)

# Mostrar los resultados
for i, (pred_num, prob) in enumerate(zip(predicciones_map, probas)):
    pred_clase = label_encoder_clasificacion.inverse_transform([pred_num])[0]
    print(f"Paciente {i+1}: Clase Predicha - {pred_clase} (Numérica: {pred_num})")
    print(f"\tProbabilidad de Enfermo (E): {prob[0]:.8f}")
    print(f"\tProbabilidad de Sano (S): {prob[1]:.8f}")


Probabilidades antes de normalizar para [ 35.9 143.    3. ]: [0.0008725019478727966, 0.0002566365904346549]
Probabilidades antes de normalizar para [ 36. 140.   2.]: [0.0007742575573920613, 0.000685094407657408]
Probabilidades antes de normalizar para [ 37.2 125.    2. ]: [1.311364363846587e-05, 0.0006681007205018544]
Probabilidades antes de normalizar para [ 36.4 120.    1. ]: [5.509246339243141e-07, 0.0008427961684071084]
Probabilidades antes de normalizar para [ 36.8 162.    3. ]: [0.0005381146778204253, 5.856247236009642e-06]
Probabilidades antes de normalizar para [ 35.9 143.    3. ]: [0.0008725019478727966, 0.0002566365904346549]
Probabilidades antes de normalizar para [ 36. 140.   2.]: [0.0007742575573920613, 0.000685094407657408]
Probabilidades antes de normalizar para [ 37.2 125.    2. ]: [1.311364363846587e-05, 0.0006681007205018544]
Probabilidades antes de normalizar para [ 36.4 120.    1. ]: [5.509246339243141e-07, 0.0008427961684071084]
Probabilidades antes de normalizar p

# Un ejemplo de clasificación de spam de un repositorio

## Se clona el repositorio

In [ ]:
# Clonar el repositorio
!git clone https://github.com/ma-shamshiri/Spam-Detector.git

# Cambiar al directorio del repositorio
%cd Spam-Detector

# Verificar el contenido del repositorio
!ls


Cloning into 'Spam-Detector'...
remote: Enumerating objects: 2317, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 2317 (delta 8), reused 4 (delta 1), pack-reused 2283 (from 1)
Receiving objects: 100% (2317/2317), 4.90 MiB | 10.12 MiB/s, done.
Resolving deltas: 100% (1170/1170), done.
/content/Spam-Detector
 gif   image   README.md  'Spam Detector'


## Funciones contenidas en spam_detector.py

In [ ]:
# Importar las bibliotecas necesarias
import os
import re
import numpy as np

# Establecer las rutas de los archivos de entrenamiento y prueba en Colab
train_path = "/content/Spam-Detector/Spam Detector/train"
test_path = "/content/Spam-Detector/Spam Detector/test"

# *** Devuelve el número total de correos electrónicos (archivos de entrenamiento) ***
def number_of_allEmails():
    counter = 0
    for directories, subdirectories, files in os.walk(train_path):
        for filename in files:
            counter += 1
    return counter

# *** Devuelve el número de correos electrónicos spam (archivos de entrenamiento) ***
def number_of_spamEmails():
    counter = 0
    for directories, subdirectories, files in os.walk(train_path):
        for filename in files:
            if "spam" in filename:
                counter += 1
    return counter

# *** Devuelve el número de correos electrónicos ham (archivos de entrenamiento) ***
def number_of_hamEmails():
    counter = 0
    for directories, subdirectories, files in os.walk(train_path):
        for filename in files:
            if "ham" in filename:
                counter += 1
    return counter

# *** Parsea el texto dado dividiéndolo por caracteres no alfabéticos ***
def text_parser(text):
    words = re.split("[^a-zA-Z]", text)
    lower_words = [word.lower() for word in words if len(word) > 0]
    return lower_words

# *** Genera las palabras del conjunto de entrenamiento: todas, spam y ham ***
def trainWord_generator():
    all_words = []
    spam_words = []
    ham_words = []
    for directories, subdirectories, files in os.walk(train_path):
        for filename in files:
            full_path = os.path.join(directories, filename)
            with open(full_path, 'r', encoding='latin-1') as target_file:
                data = target_file.read()
                words = text_parser(data)
                for word in words:
                    all_words.append(word)
                    if "ham" in filename:
                        ham_words.append(word)
                    elif "spam" in filename:
                        spam_words.append(word)
    return sorted(all_words), sorted(spam_words), sorted(ham_words)

# *** Devuelve todas las palabras únicas ***
def unique_words(all_trainWords):
    return sorted(list(set(all_trainWords)))

# *** Calcula la frecuencia de las palabras dadas ***
def frequency_calculator(words):
    wf = {}
    for word in words:
        wf[word] = wf.get(word, 0) + 1
    return wf

# *** Genera la frecuencia de cada palabra en las clases spam y ham ***
def bagOfWords_genarator(all_uniqueWords, spam_trainWords, ham_trainWords):
    spam_bagOfWords = frequency_calculator(spam_trainWords)
    ham_bagOfWords = frequency_calculator(ham_trainWords)
    for word in all_uniqueWords:
        spam_bagOfWords[word] = spam_bagOfWords.get(word, 0)
        ham_bagOfWords[word] = ham_bagOfWords.get(word, 0)
    return dict(sorted(spam_bagOfWords.items())), dict(sorted(ham_bagOfWords.items()))

# *** Genera Bag of Words suavizados para las clases spam y ham ***
def smoothed_bagOfWords(all_uniqueWords, spam_bagOfWords, ham_bagOfWords, delta):
    smoothed_spamBOW = {word: spam_bagOfWords[word] + delta for word in all_uniqueWords}
    smoothed_hamBOW = {word: ham_bagOfWords[word] + delta for word in all_uniqueWords}
    return smoothed_spamBOW, smoothed_hamBOW

# *** Calcula la probabilidad de la clase spam: P(spam) ***
def spam_probability(nb_of_allEmails, nb_of_spamEmails):
    return nb_of_spamEmails / nb_of_allEmails

# *** Calcula la probabilidad de la clase ham: P(ham) ***
def ham_probability(nb_of_allEmails, nb_of_hamEmails):
    return nb_of_hamEmails / nb_of_allEmails

# *** Calcula P(Wi|spam) para cada palabra ***
def spam_condProbability(all_uniqueWords, spam_bagOfWords, smoothed_spamBOW, delta):
    total_wf = sum(spam_bagOfWords.values()) + delta * len(all_uniqueWords)
    return {word: smoothed_spamBOW[word] / total_wf for word in smoothed_spamBOW}

# *** Calcula P(Wi|ham) para cada palabra ***
def ham_condProbability(all_uniqueWords, ham_bagOfWords, smoothed_hamBOW, delta):
    total_wf = sum(ham_bagOfWords.values()) + delta * len(all_uniqueWords)
    return {word: smoothed_hamBOW[word] / total_wf for word in smoothed_hamBOW}

# *** Genera el contenido de model.txt ***
def model_output_generator(word_numbers, words, ham_wf, ham_cp, spam_wf, spam_cp):
    output = "\n".join([f"{i+1}  {word}  {ham_wf[word]}  {ham_cp[word]}  {spam_wf[word]}  {spam_cp[word]}" for i, word in enumerate(words)])
    return output

def modelFileBuilder(model_output):
    # Guarda el archivo model.txt en la carpeta /content/
    with open("/content/model.txt", 'w', encoding='utf-8') as model_file:
        model_file.write(model_output)

# *** Devuelve los nombres de todos los correos electrónicos (archivos de prueba) ***
def get_testFileNames():
    file_names = []
    for directories, subdirectories, files in os.walk(test_path):
        for filename in files:
            file_names.append(filename)
    return file_names

# *** Devuelve las etiquetas reales (ham o spam) de cada correo electrónico ***
def get_actualLabels():
    actual_labels = []
    for directories, subdirectories, files in os.walk(test_path):
        for filename in files:
            if "ham" in filename:
                actual_labels.append("ham")
            else:
                actual_labels.append("spam")
    return actual_labels

# *** Calcula las puntuaciones de ham y spam para cada correo electrónico ***
def score_calculator(all_uniqueWords, spam_prob, ham_prob, spam_condProb, ham_condProb, delta):
    ham_scores, spam_scores = [], []
    predicted_labels, decision_labels = [], []
    for directories, subdirectories, files in os.walk(test_path):
        for filename in files:
            actual_label = "ham" if "ham" in filename else "spam"
            full_path = os.path.join(directories, filename)
            with open(full_path, encoding="latin-1") as target_file:
                email_content = target_file.read()
                email_words = text_parser(email_content)
                sigma_spamScore = sum(
                    np.log(spam_condProb[word]) for word in email_words if word in all_uniqueWords
                )
                sigma_hamScore = sum(
                    np.log(ham_condProb[word]) for word in email_words if word in all_uniqueWords
                )
                spam_score = np.log(spam_prob) + sigma_spamScore
                ham_score = np.log(ham_prob) + sigma_hamScore
                spam_scores.append(spam_score)
                ham_scores.append(ham_score)
                predicted_label = "spam" if spam_score > ham_score else "ham"
                predicted_labels.append(predicted_label)
                decision_labels.append("right" if predicted_label == actual_label else "wrong")
    return ham_scores, spam_scores, predicted_labels, decision_labels

# *** Genera el contenido de result.txt ***
def result_output_generator(fileNumbers, fileNames, predictedLabels, hamScores, spamScores, actualLabels, decisionLabels):
    output = ""
    for i in range(fileNumbers):
        output += f"{i+1}  {fileNames[i]}  {predictedLabels[i]}  {hamScores[i]}  {spamScores[i]}  {actualLabels[i]}  {decisionLabels[i]}\n"
    return output

# *** Crea el archivo result.txt ***
def resultFileBuilder(result_output):
    with open("/content/result.txt", 'w', encoding='utf-8') as model_file:
        model_file.write(result_output)

# *** Calcula métricas de precisión, recall, exactitud y F1-measure para la clase spam ***
def get_spamPrecision(fileNumbers, actualLabels, predictedLabels):
    tp = sum(1 for i in range(fileNumbers) if actualLabels[i] == "spam" and predictedLabels[i] == "spam")
    fp = sum(1 for i in range(fileNumbers) if actualLabels[i] == "ham" and predictedLabels[i] == "spam")
    return tp / (tp + fp)

def get_spamRecall(fileNumbers, actualLabels, predictedLabels):
    tp = sum(1 for i in range(fileNumbers) if actualLabels[i] == "spam" and predictedLabels[i] == "spam")
    fn = sum(1 for i in range(fileNumbers) if actualLabels[i] == "spam" and predictedLabels[i] == "ham")
    return tp / (tp + fn)

def get_spamAccuracy(fileNumbers, actualLabels, predictedLabels):
    tp_tn = sum(1 for i in range(fileNumbers) if actualLabels[i] == predictedLabels[i])
    return tp_tn / fileNumbers

def get_spamFmeasure(spam_precision, spam_recall):
    return 2 * ((spam_precision * spam_recall) / (spam_precision + spam_recall))

# *** Calcula métricas de precisión, recall, exactitud y F1-measure para la clase ham ***
def get_hamPrecision(fileNumbers, actualLabels, predictedLabels):
    tp = sum(1 for i in range(fileNumbers) if actualLabels[i] == "ham" and predictedLabels[i] == "ham")
    fp = sum(1 for i in range(fileNumbers) if actualLabels[i] == "spam" and predictedLabels[i] == "ham")
    return tp / (tp + fp)

def get_hamRecall(fileNumbers, actualLabels, predictedLabels):
    tp = sum(1 for i in range(fileNumbers) if actualLabels[i] == "ham" and predictedLabels[i] == "ham")
    fn = sum(1 for i in range(fileNumbers) if actualLabels[i] == "ham" and predictedLabels[i] == "spam")
    return tp / (tp + fn)

def get_hamAccuracy(fileNumbers, actualLabels, predictedLabels):
    tp_tn = sum(1 for i in range(fileNumbers) if actualLabels[i] == predictedLabels[i])
    return tp_tn / fileNumbers

def get_hamFmeasure(ham_precision, ham_recall):
    return 2 * ((ham_precision * ham_recall) / (ham_precision + ham_recall))

# *** Genera la tabla de resultados de evaluación ***
def evaluation_result(spam_accuracy, spam_precision, spam_recall, spam_fmeasure, ham_accuracy, ham_precision, ham_recall, ham_fmeasure):
    return (
        "################################################################################## \n"
        "#                           *** Resultados de Evaluación ***                     # \n"
        "#                                                                                # \n"
        f"#  Spam: Acc: {spam_accuracy} Prec: {spam_precision} Recall: {spam_recall} F1: {spam_fmeasure}      # \n"
        f"#   Ham: Acc: {ham_accuracy} Prec: {ham_precision} Recall: {ham_recall} F1: {ham_fmeasure}         # \n"
        "################################################################################## \n"
    )

# *** Genera el archivo evaluation.txt ***
def evaluationFileBuilder(evaluation_output):
    with open("/content/evaluation.txt", 'w', encoding='utf-8') as model_file:
        model_file.write(evaluation_output)

## Fase de entrenamiento de train.py

In [ ]:
################################################
# *************** FASE DE ENTRENAMIENTO ***************
################################################

def entrenamiento():
    print("Inicio de la fase de entrenamiento...\n")

    # Número total de correos electrónicos
    nb_of_allEmails = number_of_allEmails()
    print(f"Número total de correos electrónicos: {nb_of_allEmails}")

    # Número de correos electrónicos spam
    nb_of_spamEmails = number_of_spamEmails()
    print(f"Número de correos electrónicos spam: {nb_of_spamEmails}")

    # Número de correos electrónicos ham
    nb_of_hamEmails = number_of_hamEmails()
    print(f"Número de correos electrónicos deseados: {nb_of_hamEmails}\n")

    # Palabras del conjunto de entrenamiento: (all_trainWords, spam_trainWords, ham_trainWords)
    all_trainWords, spam_trainWords, ham_trainWords = trainWord_generator()
    print(f"Cantidad de palabras en los correos de entrenamiento: {len(all_trainWords)}")
    print(f"Cantidad de palabras únicas en correos spam: {len(set(spam_trainWords))}")
    print(f"Cantidad de palabras únicas en correos deseados: {len(set(ham_trainWords))}\n")

    # Todas las palabras únicas
    all_uniqueWords = unique_words(all_trainWords)
    print(f"Cantidad total de palabras únicas: {len(all_uniqueWords)}\n")

    # Frecuencia de cada palabra en las clases spam y ham
    spam_bagOfWords, ham_bagOfWords = bagOfWords_genarator(all_uniqueWords, spam_trainWords, ham_trainWords)
    print(f"Frecuencia total de palabras en spam: {sum(spam_bagOfWords.values())}")
    print(f"Frecuencia total de palabras en deseados: {sum(ham_bagOfWords.values())}\n")

    # Bag of Words suavizados: (smoothed_spamBOW, smoothed_hamBOW)
    smoothed_spamBOW, smoothed_hamBOW = smoothed_bagOfWords(all_uniqueWords, spam_bagOfWords, ham_bagOfWords, 0.5)
    print("Se han aplicado suavizados a las bolsas de palabras.\n")

    # Probabilidad de la clase spam: P(spam)
    spam_prob = spam_probability(nb_of_allEmails, nb_of_spamEmails)
    print(f"Probabilidad de clase spam (P(spam)): {spam_prob:.4f}")

    # Probabilidad de la clase ham: P(ham)
    ham_prob = ham_probability(nb_of_allEmails, nb_of_hamEmails)
    print(f"Probabilidad de clase deseado (P(deseado)): {ham_prob:.4f}\n")

    # P(Wi|spam) para cada palabra
    spam_condProb = spam_condProbability(all_uniqueWords, spam_bagOfWords, smoothed_spamBOW, 0.5)
    print(f"Ejemplo de P(Wi|spam) para la palabra '{list(spam_condProb.keys())[0]}': {list(spam_condProb.values())[0]:.6f}")

    # P(Wi|ham) para cada palabra
    ham_condProb = ham_condProbability(all_uniqueWords, ham_bagOfWords, smoothed_hamBOW, 0.5)
    print(f"Ejemplo de P(Wi|deseado) para la palabra '{list(ham_condProb.keys())[0]}': {list(ham_condProb.values())[0]:.6f}\n")

    # Variables de salida (para model.txt)
    word_numbers = len(all_uniqueWords)
    words = all_uniqueWords
    ham_wf = ham_bagOfWords
    ham_cp = ham_condProb
    spam_wf = spam_bagOfWords
    spam_cp = spam_condProb

    # Genera el contenido de model.txt
    model_output = model_output_generator(word_numbers, words, ham_wf, ham_cp, spam_wf, spam_cp)
    print("Contenido de model.txt generado.\n")

    # Crea el archivo model.txt
    modelFileBuilder(model_output)
    print("Archivo model.txt guardado en /content/\n")
    print("Fase de entrenamiento completada.")

In [ ]:
# Crear model.txt
entrenamiento()

Inicio de la fase de entrenamiento...

Número total de correos electrónicos: 1169
Número de correos electrónicos spam: 267
Número de correos electrónicos deseados: 902

Cantidad de palabras en los correos de entrenamiento: 747421
Cantidad de palabras únicas en correos spam: 21271
Cantidad de palabras únicas en correos deseados: 18199

Cantidad total de palabras únicas: 34022

Frecuencia total de palabras en spam: 259004
Frecuencia total de palabras en deseados: 488417

Se han aplicado suavizados a las bolsas de palabras.

Probabilidad de clase spam (P(spam)): 0.2284
Probabilidad de clase deseado (P(deseado)): 0.7716

Ejemplo de P(Wi|spam) para la palabra 'a': 0.023033
Ejemplo de P(Wi|deseado) para la palabra 'a': 0.011526

Contenido de model.txt generado.

Archivo model.txt guardado en /content/

Fase de entrenamiento completada.


## Fase de prueba de test.py

In [ ]:
# Importar tqdm para visualizar el progreso
from tqdm import tqdm

def evaluar():
    print("Inicio de la fase de prueba...\n")

    # Nombres de todos los correos electrónicos
    test_fileNames = get_testFileNames()
    print(f"Número total de correos electrónicos a probar: {len(test_fileNames)}\n")

    # Estadísticas del conjunto de entrenamiento
    nb_of_allEmails = number_of_allEmails()
    nb_of_spamEmails = number_of_spamEmails()
    nb_of_hamEmails = number_of_hamEmails()
    print(f"Número total de correos en el entrenamiento: {nb_of_allEmails}")
    print(f"Correos spam en el entrenamiento: {nb_of_spamEmails}")
    print(f"Correos deseados en el entrenamiento: {nb_of_hamEmails}\n")

    # Procesamiento del conjunto de entrenamiento
    all_trainWords, spam_trainWords, ham_trainWords = trainWord_generator()
    all_uniqueWords = unique_words(all_trainWords)
    spam_bagOfWords, ham_bagOfWords = bagOfWords_genarator(all_uniqueWords, spam_trainWords, ham_trainWords)
    smoothed_spamBOW, smoothed_hamBOW = smoothed_bagOfWords(all_uniqueWords, spam_bagOfWords, ham_bagOfWords, 0.5)
    spam_prob = spam_probability(nb_of_allEmails, nb_of_spamEmails)
    ham_prob = ham_probability(nb_of_allEmails, nb_of_hamEmails)
    spam_condProb = spam_condProbability(all_uniqueWords, spam_bagOfWords, smoothed_spamBOW, 0.5)
    ham_condProb = ham_condProbability(all_uniqueWords, ham_bagOfWords, smoothed_hamBOW, 0.5)

    print("Modelo cargado y listo para clasificar correos electrónicos.\n")

    # Procesamiento de correos de prueba
    print("Clasificando correos electrónicos...")
    ham_scores, spam_scores, predicted_labels, decision_labels = [], [], [], []

    for filename in tqdm(test_fileNames, desc="Procesando correos"):
        actual_label = "ham" if "ham" in filename else "spam"
        with open(f"{test_path}/{filename}", encoding="latin-1") as target_file:
            email_content = target_file.read()
            email_words = text_parser(email_content)

            sigma_spamScore = sum(
                np.log(spam_condProb[word]) for word in email_words if word in all_uniqueWords
            )
            sigma_hamScore = sum(
                np.log(ham_condProb[word]) for word in email_words if word in all_uniqueWords
            )

            spam_score = np.log(spam_prob) + sigma_spamScore
            ham_score = np.log(ham_prob) + sigma_hamScore

            spam_scores.append(spam_score)
            ham_scores.append(ham_score)

            predicted_label = "spam" if spam_score > ham_score else "ham"
            predicted_labels.append(predicted_label)

            decision_label = "right" if predicted_label == actual_label else "wrong"
            decision_labels.append(decision_label)

    print("\nClasificación completada.\n")

    # Resultados de evaluación
    fileNumbers = len(test_fileNames)
    result_output = result_output_generator(
        fileNumbers, test_fileNames, predicted_labels, ham_scores, spam_scores,
        get_actualLabels(), decision_labels
    )
    resultFileBuilder(result_output)
    print("Archivo result.txt generado y guardado en /content/.\n")
    # Análisis de rendimiento
    spam_precision = get_spamPrecision(fileNumbers, get_actualLabels(), predicted_labels)
    spam_recall = get_spamRecall(fileNumbers, get_actualLabels(), predicted_labels)
    spam_accuracy = get_spamAccuracy(fileNumbers, get_actualLabels(), predicted_labels)
    spam_fmeasure = get_spamFmeasure(spam_precision, spam_recall)

    ham_precision = get_hamPrecision(fileNumbers, get_actualLabels(), predicted_labels)
    ham_recall = get_hamRecall(fileNumbers, get_actualLabels(), predicted_labels)
    ham_accuracy = get_hamAccuracy(fileNumbers, get_actualLabels(), predicted_labels)
    ham_fmeasure = get_hamFmeasure(ham_precision, ham_recall)

    print("Resultados del análisis:\n")
    print(f"Spam - Precisión: {spam_precision:.4f}, Recall: {spam_recall:.4f}, Exactitud: {spam_accuracy:.4f}, F1: {spam_fmeasure:.4f}")
    print(f"Ham - Precisión: {ham_precision:.4f}, Recall: {ham_recall:.4f}, Exactitud: {ham_accuracy:.4f}, F1: {ham_fmeasure:.4f}\n")

    evaluation_result_output = evaluation_result(
        spam_accuracy, spam_precision, spam_recall, spam_fmeasure,
        ham_accuracy, ham_precision, ham_recall, ham_fmeasure
    )
    evaluationFileBuilder(evaluation_result_output)
    print("Archivo evaluation.txt generado y guardado en /content/.\n")

    print("Fase de prueba completada.")

In [ ]:
# Llama a la fase de prueba
evaluar()

Inicio de la fase de prueba...

Número total de correos electrónicos a probar: 800

Número total de correos en el entrenamiento: 1169
Correos spam en el entrenamiento: 267
Correos deseados en el entrenamiento: 902

Modelo cargado y listo para clasificar correos electrónicos.

Clasificando correos electrónicos...


Procesando correos: 100%|██████████| 800/800 [16:14<00:00,  1.22s/it]


Clasificación completada.

Archivo result.txt generado y guardado en /content/.

Resultados del análisis:

Spam - Precisión: 0.9851, Recall: 0.8250, Exactitud: 0.9062, F1: 0.8980
Ham - Precisión: 0.8495, Recall: 0.9875, Exactitud: 0.9062, F1: 0.9133

Archivo evaluation.txt generado y guardado en /content/.

Fase de prueba completada.


##Bibliografía

Shamshiri, M. (2020). Spam Detector [Código fuente]. GitHub. Recuperado de https://github.com/ma-shamshiri/Spam-Detector
